In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [9]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=3,
    lora_alpha=6,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'value', 'key', 'attention.output.dense'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 814,852 || all params: 125,463,560 || trainable%: 0.6495


In [10]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [11]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.363800,1.314823,0.751563
200,0.913800,0.377567,0.878125
300,0.379300,0.347631,0.890625
400,0.286500,0.342458,0.896875
500,0.310400,0.330961,0.901563
600,0.305100,0.323496,0.903125
700,0.308700,0.327059,0.901563
800,0.322600,0.311033,0.906250
900,0.302500,0.310219,0.901563
1000,0.290200,0.307278,0.904687


In [12]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-800'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:59<00:00,  4.18it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-800.csv


In [13]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-2000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3638,1.314823,0.387500,0.751563,0.000048,0.013405,0.048977,1.037250,-0.364063,0.515593
1,200,0.9138,0.377567,0.763393,0.878125,0.000045,0.026810,0.536233,2.420233,-0.114732,0.869344
2,300,0.3793,0.347631,0.876339,0.890625,0.000043,0.040214,0.031669,1.091100,-0.014286,0.983960
3,400,0.2865,0.342458,0.901786,0.896875,0.000040,0.053619,-0.055958,0.836599,0.004911,1.005475
4,500,0.3104,0.330961,0.896429,0.901563,0.000037,0.067024,-0.020561,0.937874,-0.005134,0.994306
5,600,0.3051,0.323496,0.896875,0.903125,0.000035,0.080429,-0.018396,0.943134,-0.006250,0.993080
6,700,0.3087,0.327059,0.899554,0.901563,0.000033,0.093834,-0.018359,0.943868,-0.002009,0.997772
7,800,0.3226,0.311033,0.893750,0.906250,0.000030,0.107239,0.011567,1.037188,-0.012500,0.986207
8,900,0.3025,0.310219,0.903125,0.901563,0.000028,0.120643,-0.007719,0.975119,0.001562,1.001733
9,1000,0.2902,0.307278,0.903571,0.904687,0.000025,0.134048,-0.017078,0.944421,-0.001116,0.998766
